In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from  sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import numpy as np
import sys
import pickle
import seaborn as sns


In [4]:
np.set_printoptions(threshold=sys.maxsize, suppress=True)

pliegues = 3

class validation_set:
	def __init__(self, X_train, y_train, X_test, y_test):
		self.X_train = X_train
		self.y_train = y_train
		self.X_test = X_test
		self.y_test = y_test

class test_set:
	def __init__(self, X_test, y_test):
		self.X_test = X_test
		self.y_test = y_test

class data_set:
	def __init__(self, validation_set, test_set):
		self.validation_set = validation_set
		self.test_set = test_set

def generate_train_test(file_name):
	pd.options.display.max_colwidth = 200

	#Lee el corpus original del archivo de entrada y lo pasa a una DataFrame
	df = pd.read_csv("breast-cancer.csv", sep=',', engine='python')
	X = df.drop(['diagnosis', 'id'],axis=1).values
	y = df['diagnosis'].values
	
	#Separa el corpus cargado en el DataFrame en el 90% para entrenamiento y el 10% para pruebas
	X_train, X_test, y_train, y_test = \
	train_test_split(X, y, test_size=0.2, shuffle = False)
	
	#~ #Crea pliegues para la validación cruzada
	validation_sets = []
	kf = KFold(n_splits=pliegues, shuffle=True)
	for train_index, test_index in kf.split(X_train):
		#print("TRAIN:", train_index, "\n",  "TEST:", test_index)
		X_train_, X_test_ = X_train[train_index], X_train[test_index]
		y_train_, y_test_ = y_train[train_index], y_train[test_index]
		#~ #Agrega el pliegue creado a la lista
		validation_sets.append(validation_set(X_train_, y_train_, X_test_, y_test_))
	
	#~ #Almacena el conjunto de prueba
	my_test_set = test_set(X_test, y_test)
	
	#~ #Guarda el dataset con los pliegues del conjunto de validación y el conjunto de pruebas
	my_data_set1 = data_set(validation_sets, my_test_set)
	
	return (my_data_set1)

my_data_set1 = generate_train_test('breast-cancer.csv')

In [19]:
puntaje = []
for x in range(pliegues):
        clf = LogisticRegression(solver = 'liblinear')
        clf.fit(my_data_set1.validation_set[x].X_train, my_data_set1.validation_set[x].y_train)

        y_pred = clf.predict(my_data_set1.validation_set[x].X_test)

        puntaje.append(accuracy_score(my_data_set1.validation_set[x].y_test, y_pred))
        print(accuracy_score(my_data_set1.validation_set[x].y_test, y_pred))
        print ('Clase real{}\nClase predicha{}'.format(my_data_set1.validation_set[x].y_test, y_pred))
        print (my_data_set1.validation_set[x].X_test)

        y_pred_proba = clf.predict_proba(my_data_set1.validation_set[x].X_test)
        print (y_pred_proba)


0.9605263157894737
Clase real['M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'B' 'B' 'M' 'M' 'M' 'M' 'B' 'M' 'M'
 'M' 'M' 'B' 'M' 'M' 'B' 'B' 'B' 'M' 'B' 'B' 'B' 'B' 'B' 'M' 'B' 'B' 'B'
 'M' 'M' 'B' 'M' 'M' 'M' 'B' 'B' 'M' 'B' 'B' 'B' 'B' 'B' 'B' 'M' 'B' 'M'
 'B' 'M' 'B' 'B' 'B' 'B' 'M' 'B' 'M' 'M' 'M' 'M' 'B' 'M' 'M' 'B' 'M' 'B'
 'M' 'B' 'B' 'M' 'B' 'M' 'B' 'M' 'B' 'B' 'M' 'M' 'B' 'M' 'B' 'M' 'M' 'M'
 'M' 'M' 'M' 'B' 'B' 'B' 'M' 'B' 'B' 'B' 'B' 'B' 'M' 'B' 'M' 'B' 'B' 'B'
 'B' 'B' 'B' 'B' 'M' 'M' 'B' 'B' 'B' 'B' 'M' 'M' 'B' 'B' 'M' 'B' 'M' 'B'
 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'M' 'B' 'B' 'B' 'B' 'B' 'B' 'M' 'B'
 'M' 'B' 'B' 'B' 'M' 'B' 'B' 'B']
Clase predicha['M' 'M' 'M' 'M' 'M' 'M' 'B' 'M' 'M' 'B' 'B' 'M' 'M' 'M' 'M' 'B' 'M' 'B'
 'M' 'M' 'B' 'M' 'M' 'B' 'M' 'B' 'M' 'B' 'B' 'B' 'B' 'B' 'M' 'B' 'B' 'B'
 'M' 'M' 'B' 'M' 'M' 'M' 'B' 'B' 'M' 'B' 'B' 'B' 'B' 'B' 'B' 'M' 'M' 'M'
 'B' 'M' 'B' 'B' 'B' 'B' 'M' 'B' 'M' 'M' 'M' 'M' 'B' 'M' 'M' 'B' 'M' 'B'
 'M' 'B' 'B' 'M' 'B' 'M' 'B' 'M' 'B' 'B' 'M' 'M

In [20]:
puntaje

[0.9605263157894737, 0.9802631578947368, 0.9470198675496688]

In [21]:
print("El puntaje mas alto en los tres pliegues es: ", max(puntaje), "El promedio de los puntajes es: ", sum(puntaje)/pliegues)

El puntaje mas alto en los tres pliegues es:  0.9802631578947368 El promedio de los puntajes es:  0.9626031137446264
